데코레이터가 무엇인지 어떻게 동작하는지 그리고 어떻게 구현되는지 살펴본다. 실제 데코레이터가 디자인을 개선하는데 어떻게 유용하게 사용될 수 있는지도 살펴본다.

이렇게 배운 내용을 토대로 이전 장에서 배운 소프트웨어 디자인의 모범 사례들을 다시 살펴보고 데코레이터가 각 원칙을 준수하는데 어떻게 도움이 되는지 살펴본다.

* 파이썬에서 데코레이터가 동작하는 방식 이해
* 함수와 크랠스에 적용되는 데코레이터를 구현하는 방법
* 일반적인 실수를 피하여 데코레이터를 효과적으로 구현하는 방법
* 데코레이터를 활용한 코드 중복을 회피(DRY 원칙)
* 데코레이터를 활용한 관심사의 분리 
* 좋은 데코레이터 사례 
* 데코레이터가 좋은 선택이 될 수 있는 일반적인 상황, 관용구, 패턴

## 파이썬의 데코레이터

파이썬에서 데코레이터는 오래 전에(PRP-318)에서 함수와 메서드의 기능을 쉽게 수정하기 위한 수단으로 소갱되었다. 

classmethod나 staticmethod 같은 함수가 원래 메서드의 정의를 변형하는데 사용되고 있었기 때문에 고안된 수단인데 이런 방법은 추가 코드가 필요하고 함수의 원래 정의를 수정해야만 했다.

다시 말해서 함수에 변형을 할 때마다 modifier 함수를 사용하여 함수를 호출한 다음 함수를 처음 정의한 것과 같은 이름으로 재할당해야 했다.

예를 들어 original 함수에 그 기능을 약간 수정한 modifier라고 하는 함수가 있는 경우 다음과 같이 작성해야 했다.

In [3]:
def original():
    return None

def modifier(ori):
    return ori()

original = modifier(original)

함수를 어떻게 동일한 이름으로 다시 할당하는지 확인해보자. 이것은 혼랍스럽고 오류가 발생하기 쉽고 번거롭다. 이런 이유로 새로운 구문이 추가되었다.

In [5]:
@modifier
def original():
    return None

즉, 데코레이터는 데코레이터 이후에 나오는 것을 데코레이터 첫 번째 파라미터로 하고 데코레이터의 결과 값을 반환하게 하는 문법적 설탕(타이필 수고 또는 읽기 쉽게 하기 위해 다른 표현으로 코딩할 수 있게 해주는 기능)일 뿐이다.

이번 예제에서 말하는 modifier는 파이썬 용어로 데코레이터라 하고, original을 데코레이팅된 함수 또는 래핑된 객체라 한다.

원래 함수와 메서드를 위해 고안되었지만 실제로는 어떤 종류의 객체에도 적용이 가능하기 때문에 여기서는 함수와 메서드, 제너레이터, 클래스에 데코레이터를 적용하는 방법을 살펴본다.

데코레이터는 래핑된 함수의 기능을 수정하고 확장하기 때문에 정확한 이름이지만 "데코레이터 디자인 패턴"과 혼동하면 안 된다.

### 함수 데코레이터

파이썬에서 데코레이터를 사용하여 기능을 변경하는 가장 간단한 방법은 함수에 적용하는 것이다. 함수에 데코레이터를 사용하면 어떤 종류의 로직이라도 적용할 수 있다. 파라미터의 유효성을 검사하거나 사전조건을 검사하거나, 기능 전체를 새롭게 정의할 수도 있고, 서명을 변경할 수도 있고, 원래 함수의 결과를 캐시하는 드으이 작업을 모두 할 수 있다.

예를 들어, 다음과 같인 도메인의 특정 예외에 대해서 특정 획수만큼 재시도하는 데코레이터를 만들어 볼 수 있다.

In [1]:
class ControlledException(Exception):
    """도메인에서 발생하는 일반적인 예외"""

def retry(operation):
    @wraps(operation)
    def wrapped(*args, **kwargs):
        last_raised = None
        RETRIES_LIMIT = 3
        for _ in range(RETRIES_LIMIT):
            try:
                return operation(*args, **kwargs)
            except ControlledException as e:
                logger.info("retrying %s", operation.__qualname__)
                last_raised = e
        raise last_raised
    return wrapped

@wrap은 지금 당장은 무시해도 된다. for 루프에서 \_는 해당 변수에 관심이 없다는 것을 뜻한다. 파이썬에서 무시해도 되는 값을 \_로 표현하는 것이 일반적인 관습이다. 

retry 데코레이터는 파라미터가 필요 없으므로 어떤 함수에도 쉽게 적용할 수 있다. 다음은 적용한 예이다.

In [3]:
@retry
def run_operation(task):
    """실행 중 예외가 발생할 것으로 예상되는 특정 작업을 실행"""
    return task.run()

앞에서 설명한 것처럼 run_operation 위에 있는 @retry는 실제로 파이썬에서 run_operation = retry(run_operation)을 실행하게 해주는 문법적 설탕일 뿐이다. 

### 클래스 데코레이터

함수에 적용한 것처럼 클래스에도 데코레이터를 사용할 수 있다(PEP-3129). 유일한 차이점은 데코레이터 함수의 파라미터로 함수가 아닌 클래스를 받는다는 점이다.

어떤 개발자들은 클래스 데코레이터가 복잡하고 가독성을 떨어뜨릴 수 있다고 말할 수 있다. 왜냐하면 클래스에서 정의한 속성과 메서드를 데코레이터 안에서 완전히 다른 용도로 변경할 수 있기 때문이다.

데코레이터를 남용할 경우 이것 또한 사실이다. 파이썬에서 보면 함수 데코레이터와 클래스 데코레이터는 다른 타입을 사용하는 것만 다를 뿐 차이점이 없다.

* 클래스 데코레이터는 코드 재사용과 DRY(Do Not Repeat Yourself) 원칙의 모든 이점을 공유한다. 클래스 데코레이터를 사용하면 여러 클래스가 특정 인터페이스(형식을 강제)나 기준을 따르도록 강제할 수 있다. 여러 클래스에 적용할 검사를 데코레이터에서 한 번만 하면 된다.

* 당장은 작고 간단한 클래스를 생성하고 나중에 데코레이터로 기능을 보강할 수 있다.

* 어떤 클래스에 대해서는 유지보수 시 데코레이터를 사용해 기존 로직을 훨씬 쉽게 변경할 수 있다. 메타클래스와 같은 방법을 사용해 보다 복잡하게 만드는 것은 주로 권장되지 않는다.

데코레이터가 유용하게 사용될 수 있는 예제를 살펴보자. 이 예제가 클래스 데코레이터의 유일한 방법은 아니며 실제로는 다양한 방법이 있을 수 있고, 각 방법마다 장단점이 있을 수 있지만 여기서는 유용성을 설명하기 위한 용도임에 유의하자.

이제 모니터링 플랫폼을 위한 이벤트 시스템은 각 이벤트의 데이터를 변환하여 외부 시스템으로 보내야한다. 그러나 각 이벤트 유형은 데이터 전송 방법에 특별한 점이 있을 수 있다.

특히 로그인 이벤트에는 자격 증명과 같은 중요한 정보를 숨겨야만 한다. timestamp와 같은 필드는 특별한 포맷으로 표시하기 때문에 변환이 필요할 수도 있다. 이러한 요구 사항을 준수하기 위한 가장 간단한 방법은 각 이벤트마다 직렬화 방법을 정의한 클래스를 만드는 것이다.

In [4]:
class LoginEventSerializer:
    def __init__(self, event):
        self.event = event
    
    def serialize(self) -> dict:
        return {
            "username": self.event.username,
            "password": "**민감한 정보 삭제**",
            "ip": self.event.ip,
            "timestamp": self.event.timestamp.strftime("%Y-%m-%d %H:%M")
        }

class LoginEvnet:
    SERIALIZER = LoginEventSerializer
    
    def __init__(self, username, password, ip, timestamp):
        self.username = username
        self.password = password
        self.ip = ip
        self.timestamp = timestamp
    
    def serialize(self) -> dict:
        return self.SERIALIZER(self).serialize()

이 방법은 처음에는 잘 동작하지만, 시간이 지나면서 시스템을 확장할수록 다음과 같은 문제가 발생하게 된다.

* 클래스가 너무 많아진다 : 이벤트 클래스와 직렬화 클래스가 1대 1로 매핑되어 있으므로 직렬화 클래스가 점점 많아지게 된다.
* 이러한 방법은 충분히 유연하지 않다 : 만약 password를 가진 다른 클래스에서도 이 필드를 숨기려면 함수로 분리한 다음 여러 클래스에서 호출해야 한다. 이는 코드를 충분히 재사용했다고 볼 수가 없다.
* 표준화 : serialize() 메서드는 모든 이벤트 클래스에 있어야만 한다. 비록 믹스인을 사용해 다른 클래스로 분리할 수 있지만 상속을 제대로 사용했다고 볼 수 없다. 

또 다른 방법은 이벤트 인스턴스와 변형 함수를 필터로 받아서 동적으로 객체를 만드는 것이다. 필터를 이벤트 인스턴스의 필드들에 적용해 직려롸하는 것이다. 이제 각 필드를 변형할 함수를 만든 다음 이들을 조합해 직렬화 객체를 만들면 된다. 

다음과 같이 클래스에 serialize() 메서드를 추가하기 위해 Serialization 객체를 활용한다.

In [34]:
from datetime import datetime

def hide_field(field) -> str:
    return "**민감한 정보 삭제**"

def format_time(field_timestamp: datetime) -> str:
    return field_timestamp.strftime("%Y-%m-%d %H:%M")

def show_original(event_field):
    return event_field

class EventSerializer:
    def __init__(self, serialization_fields: dict) -> None:
        self.serialization_fields = serialization_fields
    
    def serialize(self, event) -> dict:
        return {
            field: transformation(getattr(event, field))
            for field, transformation in self.serialization_fields.items()
        }

class Serialization:
    def __init__(self, **transformations):
        self.serializer = EventSerializer(transformations)
        
    def __call__(self, event_class):
        def serialize_method(event_instance):
            return self.serializer.serialize(event_instance)
        event_class.serialize = serialize_method
        return event_class

@Serialization(
    username = show_original,
    password = hide_field,
    ip = show_original,
    timestamp=format_time,
)
class LoginEvent:
    def __init__(self, username, password, ip, timestamp):
        self.username = username
        self.password = password
        self.ip = ip
        self.timestamp = timestamp

클래스 데코데이터에 전달된 인수를 읽는 것만으로도 username과 ip는 수정되지 않고, password 필드는 숨겨지고, timestamp는 포매팅된다는 것을 알 수 있다.

이제 개별 클래스에 serialize() 메서드를 정의하거나 믹스인을 확장할 필요도 없다. 단지 데코레이터만 추가하면 된다. 

추가로 클래스의 속성을 정의하는 것만으로 init 메서들의 로직을 구현해주는 클래스 데코레이터도 있지만 이번 예제의 범위를 벗어난다. 파이썬 3.7 이상이 버전에서의 데코레이터를 사용하면 init 함수의 탬플릿화된 단순 코드를 작성하지 않고도 다음과 같이 앞의 예제를 보다 간단하게 작성할 수 있다.

In [46]:
from datetime import datetime
loginevent = LoginEvent('were', 'were', '127.0.0.1', datetime.now())

In [48]:
loginevent.serialize()

event_instance is  <__main__.LoginEvent object at 0x068C7F50>


{'username': 'were',
 'password': '**민감한 정보 삭제**',
 'ip': '127.0.0.1',
 'timestamp': '2020-09-10 12:47'}

파이썬 3.7이상의 버전에서의 데코레이터를 사용하면 init 함수의 템플릿화된 단순 코드를 작성하지 않고도 다음과 같이 앞의 예제를 보다 간단하게 작성할 수 있다.

In [50]:
from dataclasses import dataclass
from datetime import datetime

@Serialization(
    username=show_original,
    password=hide_field,
    ip=show_original,
    timestamp=format_time,
)

@dataclass
class LoginEvent:
    username: str
    password: str
    ip: str
    timestamp: datetime

### 다른 유형의 데코레이터

이제 데코레이터의 @ 구문이 실제로 무엇을 의미하는지 알았으므로 데코레이가 단지 함수나 메서드, 크랠스에만 적용되지 않는다는 것도 알 수 있다. 사실 제너레이터나 코루틴, 심지어 이미 데코레이트된 객체도 데코레이트 가능하다. 즉 데코레이터는 스택 형태로 쌓일 수 있다.

앞의 예는 데코레이터가 어떻게 연결될 수 있는지를 보여준다. 먼저 클래스를 정의하고 @dataclass를 적용하여 속성의 컨테이너 역할을 하는 데이터 클래스로 변환한다. 그런 다음  @Serialization에서 serialize() 메서드가 추가된 새로운 클래스를 반환한다.

데코레이터의 또 다른 좋은 사용 예는 코루틴으로 사용되는 제너레이터이다. 주요 아이디어는 새로 생성된 제너레이터에 데이터를 보내기 전에 next()를 호출하여 다음 yield 문으로 넘어가야 한다는 것이다. 이러한 수작업은 모든 사용자가 기억해야 하는 것으로 에러를 유발하기 쉽다. 이런 경우 제너레이터를 파라미터로 받아 next()를 호출한 다음 다시 제너레이터를 반환하는 데코레이터를 만들면 쉽게 해결된다.

### 데코레이터에 인자 전달

지금까지 살펴본 바로도 데코레이터가 파이썬에서 강력한 도구임을 알 수 있다. 그러나 파라미터를 전달받아 로직을 추상화한다면 더욱 강력해질 수 있다. 

파라미터를 갖는 데코레이터를 구현하는 방법은 여러 가지가 있지만 가장 일반적인 방법을 살펴볼 것이다. 첫 번째는 간접 참조(indirection)를 통해 새로운 레벨의 중첩 함수를 만들어 데코레이터의 모든 것을 한 단계 더 깊게 만드는 것이다. 두 번째 방법은 데코레이터를 위한 클래스를  만드는 것이다. 

일반적으로 두 번째 방법이 가독성이 더 좋다. 왜냐하면 세 단계 이상 중첩된 클로저 함수보다 는 객체가 이해하기 쉽기 때문이다. 그러나 완벽을 기하기 위해 두 가지 모두를 살펴볼 것이며 독자는 상황에 알맞은 최선의 결정을 내리면 된다.

### 중첩 함수의 데코레이터
크게 보면 데코레이터는 함수를 파라미터로 받아서 함수를 반환하는 함수이다. 이런 함수를 고차 함수라고 부른다. 실제로는 데코레이터의 본문에 정의된 함수가 호출된다.

이제 데코레이터를 파라미터에 전달하려면 다른 수준의 간접 참조가 필요하다. 첫 번째 함수는 파라미터를 받아서 내부 함수에 전달한다. 두 번째 함수는 데코레이터가 될 함수다. 세 번째는 데코레이팅의 결과를 반환하는 함수이다. 즉 최소 세 단계의 중첩 함수가 필요하다는 뜻이다.

앞의 예제를 인스턴스마다 재시도 횟수를 지정하려고 하며 파라미터에 기본 값도 추가할 것이다. 이렇게 하려면 함수를 한 단계 더 추가해야 한다. 먼저 파라미터에 대한 것과 그리고 데코레이터 자체에 대한 것이다. 

코드는 다음과 같다.

In [51]:
@retry(arg1, arg2, ...)

@ 구문은 데코레이팅 객체에 대한 연산 결과를 반환하는 것이기 때문에 위의 코드는 의미상 다음과 같다.

In [53]:
<original_function> = retry(arg1, arg2, ...)(<original_function>)

원하는 재시도 횟수 외에도 제어하려는 예외 유형을 나타낼 수도 있다. 새 요구 사항을 반영한 새로운 코드는 다음과 같다.

In [54]:
RETRIES_LIMIT = 3
def with_retry(retires_limit=RETRIES_LIMIT, allowed_exceptions=None):
    allowed_exceptions = allowed_exceptions or (ControlledException,)
    
    def retry(operatiion):
        
        @wraps(operation)
        def wrapped(*args, **kwargs):
            last_raised = None
            for _ in range(retries_limit):
                try:
                    return operation(*args, **kwargs)
                except allowed_exceptions as e:
                    logger.info("retrying %s du to %s", operation, e)
                    last_raised = e
                raise last_raised
            return wrapped
    
    return retry

다음은 이 데코레이터를 함수에 적용한 예이다.

In [56]:
@with_retry()
def run_operation(task):
    return task.run()

@with_retry(retries_limits=5)
def run_with_custom_retries_limit(task):
    return task.run()

@with_retry(allowed_exceptions=(AttributeError,) )
def run_with_custom_exceptions(task):
    return task.run()

@with_retry(
    retries_limits =4, allowed_exceptions=(ZeroDivisionError, AttributeError)
)
def run_wit_custom_parameters(task):
    return task.run()

### 데코레이터 객체

앞의 예제에서는 세 단계의 중첩된 함수가 필요하다. 첫 번째는 데코레이터의 파라미터를 받는 함수이다. 함수 내부의 다른 함수는 이렇게 전달된 파라미터를 로직에서 사용하는 클로저이다.

이것을 보다 깔끔하게 구현하기 위해 클래스를 사용하여 데코레이터를 정의할 수 있다. 이 경우 \__init__ 메서드에 파라미터를 전달한 다음 \__call__ 이라는 매직 메서드에서 데코레이터의 로직을 구현하면 된다. 데코레이터 객체는 다음과 같다.

In [57]:
class WithRetry:
    def __init__(self, retries_limit=RETRIES_LIMIT, allowed_exceptions=None):
        self.retries_limit = retries_limit
        self.allowed_exceptions = allowed_exceptions or (ControlledException,)
    
    def __call__(self, operation):
        
        @wraps(operation)
        def wrapped(*args, **kwargs):
            last_raised = None
            for _ in range(self.retries_limit):
                try:
                    return operation(*args, **kwargs)
                except self.allowed_exceptions as e:
                    logger.info("retrying %s du to %s", operation, e)
                    last_raised = e
                raise last_raised
        return wrapped

In [59]:
@WithRetry(retries_limit=5)
def run_with_custom_limit(task):
    return task.run()

여기서 파이썬 구문이 어떻게 처리되는지 이해하는 것이 중요하다. @ 연산 전에 전달된 파라미터를 사용해 데코레이터 객체를 생성한다. 데코레이터 객체는 \__init__ 메서드에서 정해진 로직에 따라 초기화를 진행한다. 그 다음 @ 연산이 호출된다. 데코레이터 객체는 run_with_custom_retries_limit 함수를 래핑하여 \__call__ 매직 메서드를 호출한다.

\__call__ 매직 메서드는 앞의 데코레이터에서 하던 것처럼 원본 함수를 래핑하여 우리가 원하는 로직이 적용된 새로운 함수를 반환한다.

### 데코레이터 활용 우수 사례

데코레이터 활용의 우수 사례를 살펴본다. 일빈적으로 데코레이터가 좋은 선택이 될 수 있는 경우들이다.

* 파라미터 변환: 더 멋진 API를 노출하기 위해 함수의 서명을 변경하는 경우. 이 때 파라미터가 어떻게 처리되고 변환되는지를 캡술화하여 숨길 수 있다. 